XML check

In [1]:
import pandas as pd
import sqlite3
import xml.etree.ElementTree as ET
from lxml import etree
import matplotlib.pyplot as plt 
import plotly.express as px
import datetime
import os

/home/sablina/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [ ]:
df_postTriply = pd.read_csv("../dataSource-dataquality_import_export_Triply/postTriplyDataframe.csv")
df_preTriply = pd.read_csv("../dataSource-dataquality_import_export_Triply/preTriplyDataframe.csv")

# Summary Statistics

### Tellingen tijdlijn

In [6]:
import pandas as pd

csv_file = "csv/dataset_tellingen_tijdlijn.csv"

datum_vandaag = datetime.date.today().strftime("%Y-%m-%d")

dataset_tellingen_preTriply = df_preTriply.nunique().rename("Pre Triply import")
dataset_tellingen_postTriply = df_postTriply.nunique().rename("Post Triply import")

dataset_tellingen = pd.concat([dataset_tellingen_preTriply, dataset_tellingen_postTriply], axis=1)
dataset_tellingen["datum"] = datum_vandaag

dataset_tellingen = dataset_tellingen.reset_index().rename(columns={"index": "attribuut"})
cols = ["datum", "attribuut", "Pre Triply import", "Post Triply import"]
dataset_tellingen = dataset_tellingen[cols]

if os.path.exists(csv_file):
    dataset_tellingen_tijdlijn = pd.read_csv(csv_file)
    if datum_vandaag in dataset_tellingen_tijdlijn["datum"].values:
        print("dataset met datum bestaat al. Skipping save.")
    else:
        dataset_tellingen_tijdlijn = pd.concat([dataset_tellingen_tijdlijn, dataset_tellingen], ignore_index=True)
        dataset_tellingen_tijdlijn.to_csv(csv_file, index=False)
      
else:
    dataset_tellingen.to_csv(csv_file, index=False)
    print("nieuwe dataset aangemaakt!")


nieuwe dataset aangemaakt!


In [4]:

# bar chart
fig = px.bar(
    dataset_tellingen,
    x="attribuut",
    y=["Pre Triply import", "Post Triply import"],
    barmode="group",
    title="Telling van aantal per attribuut",
    labels={"value": "Aantallen", "attribuut": "Attributen"},
)

fig.update_layout(xaxis_tickangle=-90) 
fig.show()


### Tijdlijn

In [7]:
dataset_tellingen_tijdlijn = pd.read_csv("csv/dataset_tellingen_tijdlijn.csv", dtype={"datum": str})
fig = px.line(
    dataset_tellingen_tijdlijn[dataset_tellingen_tijdlijn["attribuut"] == "choId"],  # Directe filtering
    x="datum",
    y=["Pre Triply import", "Post Triply import"],
    markers=True,
    title="Verschil in 'choId' over Tijd",
    labels={"datum": "Datum", "value": "Aantal"},
)

fig.update_layout(
    xaxis=dict(type="category"),
    xaxis_tickangle=-45,
    xaxis_title="Datum",
    yaxis_title="Aantal unieke waarden",
    legend_title="Dataset",
)

fig.show()


Datakwaliteit check

In [8]:
df_postTriply_clean = df_postTriply.copy()
df_postTriply_clean["zaakId"] = pd.to_numeric(df_postTriply_clean["zaakId"], errors="coerce")
df_postTriply_clean = df_postTriply_clean[(df_postTriply_clean["zaakId"] < 6000000) | (df_postTriply_clean["zaakId"] > 7000000) & (df_postTriply_clean["zaakId"] != 2021269)]

In [9]:
df_preTriply_clean = df_preTriply.copy()
df_preTriply_clean["zaakId"] = pd.to_numeric(df_preTriply_clean["zaakId"], errors="coerce")
df_preTriply_clean = df_preTriply_clean[(df_preTriply_clean["zaakId"] < 6000000) | (df_preTriply_clean["zaakId"] > 7000000) & (df_preTriply_clean["zaakId"] != 2021269)]

In [10]:
print(df_preTriply_clean["choId"].nunique())
print(df_postTriply_clean["choId"].nunique())
print("Het verschil in aantalen choId tussen de twee datasets is: ", df_postTriply["choId"].nunique() - df_preTriply["choId"].nunique())

104269
863279
Het verschil in aantalen choId tussen de twee datasets is:  111355


In [11]:
exists = df_preTriply[(df_preTriply["zaakId"] >= 6000000) & (df_preTriply["zaakId"] <= 7000000)].any().any()

if exists:
    print("De dataset bevat zaakId tussen 6,000,000 and 7,000,000.")
else:
    print("Geen zaakId gevonden in range 6,000,000 to 7,000,000.")


Geen zaakId gevonden in range 6,000,000 to 7,000,000.
